# Path planning
The goal of the path planning is to find the shortest path for Santo to travel to all houses. It takes as input the visibility graph of the map, and returns the optimal path, which is a list of coordinates for Santo to follow. This problem is divided in 3 parts:
* Simplifiying the graph
* Solving a Travelling Salesman Problem
* Reconstructing the optimal path

Let's see how it works with an example:

![title](vis_graph.png)

### Simplifying the graph

We want to simplify the graph by only keeping the indices of Santo and of the targets. We also want all of them to be connected. To do that, we will apply the dijkstra algorithm multiple times, to find the distance (and the path, used later) between each pair of points of interest (here points of interest are the start point and the target points).

For the example above, we take a visibility graph where the start index is 4, and the targets indices are 0 and 3. We want the distance between the vertices 4 and 0, 4 and 3, 0 and 3.

The function "create_distance_path_matrix" stores this information in matrices:

Input:
* Visibility graph
* Start index
* Target indices

Output:
* Matrix storing the distance between all points of interest
* Matrix storing the path between all points of interest

We can divide by two the number of calculations if we use the fact that the distance from index "x" to "y" is the same as from "y" to "x", and that the path is just reversed.

In [1]:
def create_distance_path_matrix(visibility_graph,start_idx,targets_idx_list):
    #we treat start as an objective, from which we start
    targets_idx_list.insert(0, start_idx)

    nb_interest_point = len(targets_idx_list)
    distance_matrix = np.zeros((nb_interest_point, nb_interest_point))
    path_matrix = np.zeros((nb_interest_point, nb_interest_point), dtype=object)
    for i in range(nb_interest_point):
        for j in range(nb_interest_point):
            if i==j:
                distance_matrix[i][j] = 0
                path_matrix[i][j] = [[targets_idx_list[i]]]
            elif i > j:
                distance_matrix[i][j], path_matrix[i][j] = dijkstra_aglorithm(targets_idx_list[i], targets_idx_list[j], visibility_graph)
                distance_matrix[j][i] = distance_matrix[i][j]
                path_matrix[j][i] = path_matrix[i][j][::-1]


    return distance_matrix, path_matrix

From the example above, the output of the function "create_distance_path_matrix" would be:

![title](path_matrices.png)

so the simplified graph can be represented as:

![title](simple_graph.png)

### Solving a Travelling Salesman Problem
The next part is to solve a problem similar to the Travelling Salesman Problem, except that it Santo doesn't need to return to the starting point. We figured that in our project, we will most likely have 3 or 4 targets, so we don't need a fancy algorithm and can afford to test all possible solutions. In fact, even with 8 targets we would need to solve 8! = 40'320 solutions which isn't that much for a modern computer.

We also notice that because all points are connceted to each other, we don't need a graph representation and can just use the distance matrix calulated in the "Simplifying the graph" part.

The "shortest_path" function is a recursive function, and has the following signature:

Input:
* The starting index
* The list of already visited indices (this list only contains the starting index on the first call)
* The distance matrix calulated previously

Output:
* The shortest distance to go through all indices that aren't yet visited
* The index order to follow to have this shortest distance

In [2]:
def shortest_path(start_local_idx, visited_idx_list, distance_array):
    nb_points = np.size(distance_array,0) #number of points to go through, + 1 for the starting point

    distance = np.Inf #value to optimize
    idx_list = np.zeros(np.size(distance_array,0)) #saves optimal path

    if np.size(visited_idx_list) == nb_points : #end of recursion, all points have been passed through
        return 0, np.zeros(np.size(distance_array,0))

    temp_distance = 0
    temp_idx = np.zeros(np.size(distance_array,0))

    for i in range(nb_points):
        if i in visited_idx_list : #tries all points that haven't been visited
            continue
        temp_distance, temp_idx = shortest_path(i, np.append(visited_idx_list, i), distance_array)
        temp_distance += distance_array[start_local_idx][i]
        
        if temp_distance < distance : #if found a better way, updates its optimal path and value
            distance = temp_distance
            idx_list = temp_idx
            idx_list[visited_idx_list.size] = i

    return (distance, idx_list)

From the example above, as we only have 2 targets, we need to test 2! = 2 possibilities:
* The order 4-0-3 which from the distance matrix would give dist(4,0) + dist(0,3) = 4 + 3 = 7
* The order 4-3-0 which from the distance matrix would give dist(4,3) + dist(3,0) = 5 + 3 = 8

As the first order 4-0-3 gives a shorter distance, we remember this sequence.

### Reconstructing the optimal path
Now that we know the order in which to go to the targets, we reconstruct the optimal path.

The signature of the functions is the following:

Input:
* The path matrix calculated in the "create_distance_path_matrix" function
* The targets index order given by the "shortest_path" function
* Vertices array which gives the coordinates based on the vertex index

Output:
* The list of coordinates to follow to go through all targets in the shortest path

In [4]:
def reconstruct_optimal_path(path_array, targets_idx_order, vertices):
    optimal_path = []
    for i in range(len(targets_idx_order)-1):
        for j in range(len(path_array[int(targets_idx_order[i])][int(targets_idx_order[i+1])])): #use the path matrix to reconstruct
            if i != 0 and j == 0: #to avoid repetition of the same index when joining paths, as the end index of a segement is the start index of the following
                continue
            optimal_path.append(vertices[path_array[int(targets_idx_order[i])][int(targets_idx_order[i+1])][j]]) #use vertices array which gives coordinates based on index
    return optimal_path

In the example above, we know that the target order is 4-0-3, so we retrieve the path to travel 4-0 then 0-3. From the Path Matrix, it is [4,1,0] and [0,2,3], so by following the indices [4,1,0,2,3] in the visibility graph, we have the shortest path.